# Fit models to human data

In [13]:
import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

In [14]:
from Config.config import PATHS
from Classes.parameter_recovery import ParameterFit
from Classes.cognitive_model_agents import MODELS

### Load human data

In [15]:
# Load data into a dataframe

two_player = True
if two_player:
	file_name = '2-player-UR.csv' # <= Data from 2-player UR experiment
	best_fits_file = PATHS['parameter_fit_results'] / Path('best_fit_2P.json')
else:
	file_name = 'multi-player.csv' # <= Data from multi-player IU experiment
	best_fits_file = PATHS['parameter_fit_results'] / Path('best_fit_MP.json')

data_folder = PATHS['human_data']
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')

data = pd.read_csv(file)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head(2)

Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/2-player-UR.csv...


,group,round,player,score,threshold,num_players,decision
0,Grupo-0001,1,405186428721292,0,0.5,2,0
1,Grupo-0001,1,728608683559397,1,0.5,2,1


### Select models

In [24]:
# check_out_these = [model.name() for model in MODELS]
# check_out_these = ['Payoff-M2', 'Fairness-M2', 'AvailableSpace-M2']
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
# check_out_these = ['Payoff-M2', 'Fairness-M2']
# check_out_these = ['Fairness-M2']
# check_out_these = ['FRA']
check_out_these = ['FRA+Payoff+Attendance']
my_models = [model for model in MODELS if model.name() in check_out_these]

### Fit with scipy

In [25]:
ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_Scipy.json',
    optimizer_type='scipy',
    hyperparameters=None,
    new_file=True
)

Fitting models...:   0%|          | 0/1 [00:00<?, ?it/s]

Fitting data to model FRA+Payoff+Attendance...
Creating parameter recovery class...
Running optimizer...


### Fit with Bayesian Optimizer

In [26]:
hyperparameters = {
    'init_points':128,
    'n_iter':64
}

ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_Bayesian.json',
    optimizer_type='bayesian',
    hyperparameters=hyperparameters,
    new_file=True
)

Fitting models...:   0%|          | 0/1 [00:00<?, ?it/s]

Fitting data to model FRA+Payoff+Attendance...
Creating parameter recovery class...
Running optimizer...
|   iter    |  target   | invers... |   bias    | learni... | len_hi... |     c     |   delta   |
-------------------------------------------------------------------------------------------------
| 1         | -3171.770 | 27.272386 | 0.7203244 | 0.0001143 | 1.6046651 | 0.5733779 | 0.0184677 |
| 2         | -4336.337 | 12.734393 | 0.3455607 | 0.3967674 | 2.0776334 | 0.7095972 | 0.1370439 |
| 3         | -2308.387 | 13.880491 | 0.8781174 | 0.0273875 | 2.3409350 | 0.7086524 | 0.1117379 |
| 4         | -6497.718 | 9.8443771 | 0.1981014 | 0.8007445 | 2.9365231 | 0.6567120 | 0.1384645 |
| 5         | -3832.221 | 56.212516 | 0.8946066 | 0.0850442 | 1.0781095 | 0.5849152 | 0.1756285 |
| 6         | -4358.753 | 7.1958505 | 0.4211076 | 0.9578895 | 2.0663305 | 0.8459385 | 0.0631031 |
| 7         | -4625.488 | 44.249558 | 0.8346256 | 0.0182882 | 2.5002886 | 0.9944305 | 0.1496331 |
| 8         |

### Keep best fit from both optimizers

In [27]:
df_2P_Scipy = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_Scipy.json', lines=True)
df_2P_Bayesian = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_Bayesian.json', lines=True)

df = pd.merge(left=df_2P_Scipy, right=df_2P_Bayesian, on='model_name', how='outer')

df_list = []
for model in my_models:
    df1 = df[df['model_name'] == model.name()]
    deviance_x = -df1['deviance_x'].unique()[0]
    deviance_y = -df1['deviance_y'].unique()[0]
    if deviance_x < deviance_y:
        df1.drop(columns=[col for col in df1.columns if '_y' in col], inplace=True)
        df1.rename(columns={col: col.replace('_x', '') for col in df1.columns}, inplace=True)
    else:
        df1.drop(columns=[col for col in df1.columns if '_x' in col], inplace=True)
        df1.rename(columns={col: col.replace('_y', '') for col in df1.columns}, inplace=True)
    df_list.append(df1)
df = pd.concat(df_list, ignore_index=True)

In [28]:
df

,model_name,model,fixed_parameters,free_parameters,deviance,AIC
0,FRA+Payoff+Attendance,FRAplus,"{'num_agents': 2, 'threshold': 0.5}","{'inverse_temperature': 5.623843229658409, 'bi...",-2179.451475,4370.90295


### Save on file

In [ ]:
new_file = False

if new_file:
    df.to_json(best_fits_file, orient='records', lines=True)
else:
    list_fixed_parameters_new = df['fixed_parameters'].values
    best_fit = pd.read_json(best_fits_file, lines=True)
    for fixed_parameters_new in list_fixed_parameters_new:
        df1 = df[df['fixed_parameters'] == fixed_parameters_new]
        for model in my_models:
            df2 = df1[df1['model_name'] == model.name()]
            if model.name() not in best_fit['model_name'].values:
                best_fit = pd.concat([best_fit, df1], ignore_index=True)
            else:
                best_fit.loc[best_fit['model_name'] == model.name(), df2.columns] = df2.values

    best_fit.to_json(best_fits_file, orient='records', lines=True)

---

In [ ]:
fixed_parameters_new = df['fixed_parameters'].values[0]
fixed_parameters_old = best_fit['fixed_parameters'].values[0]

In [38]:
fixed_parameters_old == fixed_parameters_new

True